## Demo Debezuim

### Шаг 1

На БД `postgres` выполняем
```sql
select * from pg_stat_replication;
```

Видим, что ответ пустой; Значит, сейчас репликации нет;

Далее выполняем
```sql
create table public.customers (
    id int,
    name text
);

alter table public.customers replica identity full;

insert into public.customers (id, name)
values
    (1, 'ilya'), (2, 'kate'), (3, 'ivan')
;
```

Это будут наши стартовые значения

### Шаг 2

Настройка debezium

In [1]:
!rm connector.json

rm: connector.json: No such file or directory


In [3]:
import json

config = {
  "name": "pg-connector",
  "config": {
    "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
    "plugin.name": "pgoutput",
    
    "database.hostname": "postgres",
    "database.port": "5432",
    "database.user": "postgres",
    "database.password": "postgres",
    "database.dbname" : "postgres",
    "database.server.name": "pg-dev",
    
    "table.include.list": "public.(.*)",
    "heartbeat.interval.ms": "5000",
    "slot.name": "dbname_debezium",
    "publication.name": "dbname_publication",
    
    "transforms": "AddPrefix",
    "transforms.AddPrefix.type": "org.apache.kafka.connect.transforms.RegexRouter",
    "transforms.AddPrefix.regex": "pg-dev.public.(.*)",
    "transforms.AddPrefix.replacement": "data.cdc.dbname",
      
    "topic.creation.default.cleanup.policy": "delete",
    "topic.creation.default.partitions": "1",
    "topic.creation.default.replication.factor": "1",
    "topic.creation.default.retention.ms": "604800000",
    "topic.creation.enable": "true",
    "topic.prefix": "postgres"
  }
}

with open('connector.json', 'w') as f:
    json.dump(config, f)

In [1]:
import json
topic = 'postgres.public.aircraft'
record_keys = 'aircraft_code'
config = {
            "name": topic.replace('.', '-'),
            "config": {
                "connector.class": "io.debezium.connector.jdbc.JdbcSinkConnector",
                "topics": topic,
                "connection.url": "jdbc:postgresql://postgres_dwh:5432/postgres",
                "connection.username": "postgres",
                "connection.password": "postgres",
                "tasks.max":"1",
                "insert.mode": "upsert",
                "delete.enabled": "false",
                "primary.key.mode": "record_key",
                "primary.key.fields": record_keys,
                "schema.evolution": "basic"
                }
            }

with open('connector2.json', 'w') as f:
    json.dump(config, f)

In [2]:
!curl -X POST --location "http://localhost:8083/connectors" -H "Content-Type: application/json" -H "Accept: application/json" -d @connector2.json

{"name":"postgres-public-aircraft","config":{"connector.class":"io.debezium.connector.jdbc.JdbcSinkConnector","topics":"postgres.public.aircraft","connection.url":"jdbc:postgresql://postgres_dwh:5432/postgres","connection.username":"postgres","connection.password":"postgres","tasks.max":"1","insert.mode":"upsert","delete.enabled":"false","primary.key.mode":"record_key","primary.key.fields":"aircraft_code","schema.evolution":"basic","name":"postgres-public-aircraft"},"tasks":[],"type":"sink"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   957  100   496  100   461  15466  14374 --:--:-- --:--:-- --:--:-- 29906


Подключение создалось;<br>
Чтобы дополнительно в этом убедиться, можем посмотреть в UI: http://localhost:8080

### Шаг 3

Смотрим, что топик в kafka создался

In [7]:
!rm clusters.json

rm: clusters.json: No such file or directory


In [5]:
# Нас интересует cluster_id

!curl http://localhost:8082/v3/clusters -o clusters.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1001    0  1001    0     0   5764      0 --:--:-- --:--:-- --:--:--  5786


In [6]:
with open('clusters.json', 'r') as f:
    clusters = json.load(f)
clusters

{'kind': 'KafkaClusterList',
 'metadata': {'self': 'http://rest-proxy:8082/v3/clusters', 'next': None},
 'data': [{'kind': 'KafkaCluster',
   'metadata': {'self': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg',
    'resource_name': 'crn:///kafka=dYcsmY32SRecmP5353bVlg'},
   'cluster_id': 'dYcsmY32SRecmP5353bVlg',
   'controller': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/brokers/1'},
   'acls': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/acls'},
   'brokers': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/brokers'},
   'broker_configs': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/broker-configs'},
   'consumer_groups': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/consumer-groups'},
   'topics': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics'},
   'partition_reassignments': {'related': 'http://rest-proxy:8082/v3/

Запоминаем cluster_id = `TCJFECM_Stio2wfa268A5g`

In [13]:
!rm topics.json

rm: topics.json: No such file or directory


In [3]:
import json

In [17]:
!curl http://localhost:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/postgres.public.boarding_passes/configs -o 1.json

with open('1.json', 'r') as f:
    one = json.load(f)
one

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 15735    0 15735    0     0   231k      0 --:--:-- --:--:-- --:--:--  232k


{'kind': 'KafkaTopicConfigList',
 'metadata': {'self': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/postgres.public.boarding_passes/configs',
  'next': None},
 'data': [{'kind': 'KafkaTopicConfig',
   'metadata': {'self': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/postgres.public.boarding_passes/configs/cleanup.policy',
    'resource_name': 'crn:///kafka=dYcsmY32SRecmP5353bVlg/topic=postgres.public.boarding_passes/config=cleanup.policy'},
   'cluster_id': 'dYcsmY32SRecmP5353bVlg',
   'name': 'cleanup.policy',
   'value': 'delete',
   'is_read_only': False,
   'is_sensitive': False,
   'source': 'DYNAMIC_TOPIC_CONFIG',
   'synonyms': [{'name': 'cleanup.policy',
     'value': 'delete',
     'source': 'DYNAMIC_TOPIC_CONFIG'},
    {'name': 'log.cleanup.policy',
     'value': 'delete',
     'source': 'DEFAULT_CONFIG'}],
   'topic_name': 'postgres.public.boarding_passes',
   'is_default': False},
  {'kind': 'KafkaTopicConfig',
   'metadata': {'self

In [ ]:
!curl http://localhost:8082/v3/clusters/x7CHkIiVTgiq-_6Ob3dE_A/topics -o topics.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: rest-proxy


In [8]:
with open('topics.json', 'r') as f:
    topics = json.load(f)
topics

{'kind': 'KafkaTopicList',
 'metadata': {'self': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics',
  'next': None},
 'data': [{'kind': 'KafkaTopic',
   'metadata': {'self': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/__debezium-heartbeat.postgres',
    'resource_name': 'crn:///kafka=dYcsmY32SRecmP5353bVlg/topic=__debezium-heartbeat.postgres'},
   'cluster_id': 'dYcsmY32SRecmP5353bVlg',
   'topic_name': '__debezium-heartbeat.postgres',
   'is_internal': False,
   'replication_factor': 1,
   'partitions_count': 1,
   'partitions': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/__debezium-heartbeat.postgres/partitions'},
   'configs': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/__debezium-heartbeat.postgres/configs'},
   'partition_reassignments': {'related': 'http://rest-proxy:8082/v3/clusters/dYcsmY32SRecmP5353bVlg/topics/__debezium-heartbeat.postgres/partitions/-/reassignment'},
   

In [14]:
for i in topics['data']:
    # print(i)
    # break
    print(i['topic_name'])

__debezium-heartbeat.postgres
_schemas
connect_configs
connect_offsets
connect_statuses
postgres.public.aircraft
postgres.public.airports
postgres.public.boarding_passes
postgres.public.bookings
postgres.public.flights
postgres.public.seats
postgres.public.ticket_flights
postgres.public.tickets


### Шаг 3

Видим, что при init для таблицы, в которой у нас уже были строки, уже создался топик<br>
Давайте посмотрим, что туда записалось

In [21]:
import json
from kafka import KafkaConsumer

KAFKA_HOST = "localhost"
KAFKA_CONSUMER_GROUP = "backend"

def read_single_message(topic=None):
    assert topic is not None, 'You must specify topic name'
    
    consumer = KafkaConsumer(
        bootstrap_servers=f"{KAFKA_HOST}:9092",
        value_deserializer=lambda v: v if v is None else json.loads( v.decode("utf-8") ),
        auto_offset_reset="earliest",
        group_id=KAFKA_CONSUMER_GROUP
    )
    consumer.subscribe(topics=topic)

    try:
        for message in consumer:
            value = message.value
            return value
    except Exception as e:
            print("Closing consumer due to error\n")
            consumer.close()
            raise e
    finally:
        print("Closing consumer due to finish\n")
        consumer.close()

In [23]:
value = read_single_message('postgres.public.customers')
value

Closing consumer due to finish



{'schema': {'type': 'struct',
  'fields': [{'type': 'struct',
    'fields': [{'type': 'int32', 'optional': True, 'field': 'id'},
     {'type': 'string', 'optional': True, 'field': 'name'}],
    'optional': True,
    'name': 'postgres.public.customers.Value',
    'field': 'before'},
   {'type': 'struct',
    'fields': [{'type': 'int32', 'optional': True, 'field': 'id'},
     {'type': 'string', 'optional': True, 'field': 'name'}],
    'optional': True,
    'name': 'postgres.public.customers.Value',
    'field': 'after'},
   {'type': 'struct',
    'fields': [{'type': 'string', 'optional': False, 'field': 'version'},
     {'type': 'string', 'optional': False, 'field': 'connector'},
     {'type': 'string', 'optional': False, 'field': 'name'},
     {'type': 'int64', 'optional': False, 'field': 'ts_ms'},
     {'type': 'string',
      'optional': True,
      'name': 'io.debezium.data.Enum',
      'version': 1,
      'parameters': {'allowed': 'true,last,false,incremental'},
      'default': 'fa

Что у нас тут получилось:
1) Есть схема сообщения
2) Так как все строки новые, то `before` содержит `None`
3) В то же время, `after` содержит значения в строке

Посмотрим, что происходит при insert
Выполним на БД следующий скрипт
```sql
insert into public.customers (id, name)
values
    (4, 'david')
;
```

In [25]:
for i in range(2):
    read_single_message('postgres.public.customers')

Closing consumer due to finish

Closing consumer due to finish



In [27]:
value = read_single_message('postgres.public.customers')
value['payload']

Closing consumer due to finish



{'before': None,
 'after': {'id': 4, 'name': 'david'},
 'source': {'version': '2.2.0.Alpha3',
  'connector': 'postgresql',
  'name': 'postgres',
  'ts_ms': 1729014334072,
  'snapshot': 'false',
  'db': 'postgres',
  'sequence': '[null,"22972784"]',
  'schema': 'public',
  'table': 'customers',
  'txId': 491,
  'lsn': 22972784,
  'xmin': None},
 'op': 'c',
 'ts_ms': 1729014334218,
 'transaction': None}

Получили все то же самое<br>
Теперь посмотрим на update
```sql
update public.customers
set name = 'diana'
where id = 4;
```

In [29]:
value = read_single_message('postgres.public.customers')
value['payload']

Closing consumer due to finish



{'before': {'id': 4, 'name': 'david'},
 'after': {'id': 4, 'name': 'diana'},
 'source': {'version': '2.2.0.Alpha3',
  'connector': 'postgresql',
  'name': 'postgres',
  'ts_ms': 1729014380110,
  'snapshot': 'false',
  'db': 'postgres',
  'sequence': '["22973104","22973160"]',
  'schema': 'public',
  'table': 'customers',
  'txId': 492,
  'lsn': 22973160,
  'xmin': None},
 'op': 'u',
 'ts_ms': 1729014380246,
 'transaction': None}

Что мы имеем:
1) `before` содержит старые значения в строке
2) `after` содержит новые значения в строке

Посмотрим, что произойдет при delete
```sql
delete
from public.customers
where id = 4;
```

In [31]:
value = read_single_message('postgres.public.customers')
value['payload']

Closing consumer due to finish



{'before': {'id': 4, 'name': 'diana'},
 'after': None,
 'source': {'version': '2.2.0.Alpha3',
  'connector': 'postgresql',
  'name': 'postgres',
  'ts_ms': 1729014420617,
  'snapshot': 'false',
  'db': 'postgres',
  'sequence': '["22973304","22973360"]',
  'schema': 'public',
  'table': 'customers',
  'txId': 493,
  'lsn': 22973360,
  'xmin': None},
 'op': 'd',
 'ts_ms': 1729014421006,
 'transaction': None}

Что мы имеем:
1) `before` содержит старые значения в строке
2) `after` содержит `None`

<b>ВАЖНО:</b> Debezium [не умеет](https://github.com/MaterializeInc/materialize/issues/6596#issuecomment-963479859) в обработку truncate

### Шаг 4

Посмотрим, что произойдет при создании новой таблицы
```sql
create table public.stores (
    id int,
    city text
);
ALTER TABLE public.stores REPLICA IDENTITY FULL;
```

In [45]:
!rm topics.json

In [47]:
!curl http://localhost:8082/v3/clusters/x7CHkIiVTgiq-_6Ob3dE_A/topics -o topics.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5452    0  5452    0     0   123k      0 --:--:-- --:--:-- --:--:--  123k


In [49]:
with open('topics.json', 'r') as f:
    topics = json.load(f)

for i in topics['data']:
    print(i['topic_name'])

__debezium-heartbeat.postgres
_schemas
connect_configs
connect_offsets
connect_statuses
postgres.public.customers
service_backend_feed


Видим, что топика нет<br>
Все потому, что таблица пустая<br>
Добавим в нее строк
```sql
insert into public.stores (id, city)
values
    (1, 'Moscow'),
    (2, 'London'),
    (3, 'Milano')
;
```

In [51]:
!rm topics.json
!curl http://localhost:8082/v3/clusters/x7CHkIiVTgiq-_6Ob3dE_A/topics -o topics.json
        
with open('topics.json', 'r') as f:
    topics = json.load(f)

for i in topics['data']:
    print(i['topic_name'])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6234    0  6234    0     0  58302      0 --:--:-- --:--:-- --:--:-- 58811
__debezium-heartbeat.postgres
_schemas
connect_configs
connect_offsets
connect_statuses
postgres.public.customers
postgres.public.stores
service_backend_feed


Теперь топик есть, и его можно почитать

In [53]:
for i in range(3):
    value = read_single_message('postgres.public.stores')
    print(value['payload'], end='\n---\n')

Closing consumer due to finish

{'before': None, 'after': {'id': 1, 'city': 'Moscow'}, 'source': {'version': '2.2.0.Alpha3', 'connector': 'postgresql', 'name': 'postgres', 'ts_ms': 1729014541776, 'snapshot': 'false', 'db': 'postgres', 'sequence': '["23088648","23088704"]', 'schema': 'public', 'table': 'stores', 'txId': 496, 'lsn': 23088704, 'xmin': None}, 'op': 'c', 'ts_ms': 1729014541922, 'transaction': None}
---
Closing consumer due to finish

{'before': None, 'after': {'id': 2, 'city': 'London'}, 'source': {'version': '2.2.0.Alpha3', 'connector': 'postgresql', 'name': 'postgres', 'ts_ms': 1729014541776, 'snapshot': 'false', 'db': 'postgres', 'sequence': '["23088648","23088776"]', 'schema': 'public', 'table': 'stores', 'txId': 496, 'lsn': 23088776, 'xmin': None}, 'op': 'c', 'ts_ms': 1729014541923, 'transaction': None}
---
Closing consumer due to finish

{'before': None, 'after': {'id': 3, 'city': 'Milano'}, 'source': {'version': '2.2.0.Alpha3', 'connector': 'postgresql', 'name': 'pos